# Simple CellML example


This notebook creates a simple model in [CellML version 2](https://www.cellml.org/specifications/cellml_2.0/cellml_2_0_normative_specification.pdf) for illustration.  It is based on the libCellML tutorial series which can found [here](https://libcellml.org/documentation/v0.4.0/user/tutorials/index).

## 1) Including libraries

In [23]:
from libcellml import Analyser, Component, Model, Printer, Units, Validator, Variable, cellmlElementTypeAsString
from combine_notebooks.cellml_utilities import print_model, print_issues
from combine_notebooks import RESULTS_DIR

## 2) Declaring the CellML model

The first step is to create a Model item which will later contain the component and the units it needs.

In [24]:
model = Model()

Each CellML element must have a name, which is set using the setName() function.

In [25]:
model.setName('tutorial_1_model')
model.setId('tutorial_1_model_id_is_here')

We'll create a wrapper component whose only job is to encapsulate the other components. This makes is a lot easier for this model to be reused, as the connections between components internal to this one won't need to be re-established. Note that the constructor for all named CellML entities is overloaded, so you can pass it the name string at the time of creation.

In [26]:
component = Component('i_am_a_component')
component.setId('my_component_id')

Finally we need to add the component to the model.  This sets it at the top-level of 
the components' encapsulation hierarchy.  All other components need to be added 
to this component, rather than the model.

In [27]:
model.addComponent(component)

True

 In order to actually model anything, we need to include the mathematical equations which represent the physical situation of interest.  To do this we will start by creating the variables $a$, $b$, and $c$.

In [28]:
a = Variable("a")
a.setUnits("dimensionless")
a.setInitialValue("b")
a.setInterfaceType("public_and_private")
component.addVariable(a)

b = Variable("b")
b.setUnits("dimensionless")
b.setInterfaceType("public_and_private")
component.addVariable(b)

c = Variable("c")
c.setUnits("dimensionless")
c.setInterfaceType("public_and_private")
component.addVariable(c)

print_model(model)

MODEL: 'tutorial_1_model', id: 'tutorial_1_model_id_is_here'
    UNITS: 0 custom units
    COMPONENTS: 1 components
        [0]: 'i_am_a_component', id: 'my_component_id'
            VARIABLES: 3 variables
                [0]: a [dimensionless], initial = b
                [1]: b [dimensionless]
                [2]: c [dimensionless]


Equations are modelled using MathML2 strings.  Here we are create MathML to model the simple equation ; $ a = b + c $.

In [29]:
# Add the header and footer strings.
math_header = '<math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">'
math_footer = '</math>'

# Create the MathML2 string representing the governing equations.  
equation1 = \
    "   <apply>"\
    "       <eq/>"\
    "       <ci>a</ci>"\
    "       <apply>"\
    "           <plus/>"\
    "           <ci>b</ci>"\
    "           <ci>c</ci>"\
    "       </apply>"\
    "   </apply>"

Include the MathML strings in the component.

In [30]:
component.setMath(math_header)
component.appendMath(equation1)
component.appendMath(math_footer)

## 3) Write, print and validate the generated model

Print the model to using the print_model helper function and 
check it is what you'd expect.

In [31]:
print_model(model)

MODEL: 'tutorial_1_model', id: 'tutorial_1_model_id_is_here'
    UNITS: 0 custom units
    COMPONENTS: 1 components
        [0]: 'i_am_a_component', id: 'my_component_id'
            VARIABLES: 3 variables
                [0]: a [dimensionless], initial = b
                [1]: b [dimensionless]
                [2]: c [dimensionless]


Create a validator and use it to check the model for issues.

In [32]:
validator = Validator()
validator.validateModel(model)
print_issues(validator)

Recorded 0 issues!



Save the model to a .cellml file and print the raw cellml text.

In [36]:
printer = Printer()
serialised_model = printer.printModel(model)
# Write the serialised string to a file.
file_loc = str(RESULTS_DIR) + "/hello_world_cellml.cellml"
write_file = open(file_loc, "w")
write_file.write(serialised_model)
xml = open(file_loc).read()
print(xml)

<?xml version="1.0" encoding="UTF-8"?>
<model xmlns="http://www.cellml.org/cellml/2.0#" name="tutorial_1_model" id="tutorial_1_model_id_is_here">
  <component name="i_am_a_component" id="my_component_id">
    <variable name="a" units="dimensionless" initial_value="b" interface="public_and_private"/>
    <variable name="b" units="dimensionless" interface="public_and_private"/>
    <variable name="c" units="dimensionless" interface="public_and_private"/>
    <math xmlns="http://www.w3.org/1998/Math/MathML" xmlns:cellml="http://www.cellml.org/cellml/2.0#">
      <apply>
        <eq/>
        <ci>a</ci>
        <apply>
          <plus/>
          <ci>b</ci>
          <ci>c</ci>
        </apply>
      </apply>
    </math>
  </component>
</model>



## 4) Simulating the model